<a href="https://colab.research.google.com/github/prime29haruno/Kaggle_Titanic/blob/main/Titanic9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd, os

# 1) データ読み込み
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test  = pd.read_csv("/kaggle/input/titanic/test.csv")

# 2) 前処理（Age の欠損だけ中央値で埋めればOK）
train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"]  = test["Age"].fillna(train["Age"].median())

# 3) 予測関数（2案）
def predict_A(df):
    # 性別のみ：女性=1, 男性=0
    return (df["Sex"] == "female").astype(int)

def predict_B(df):
    # 性別 + 年齢 + Pclass：優しめの改良
    out = pd.Series(0, index=df.index)              # まず全員0(死亡)
    out[df["Sex"] == "female"] = 1                  # 女性は生存
    # 3等・女性・40歳以上は死亡に戻す（過学習しにくい軽い補正）
    mask_f_bad = (df["Sex"]=="female") & (df["Pclass"]==3) & (df["Age"]>=40)
    out[mask_f_bad] = 0
    # 1等・男性・10歳以下は生存
    mask_m_kid = (df["Sex"]=="male") & (df["Pclass"]==1) & (df["Age"]<=10)
    out[mask_m_kid] = 1
    return out.astype(int)

# 4) train で検証して良い方を選ぶ
train_pred_A = predict_A(train)
train_pred_B = predict_B(train)
acc_A = (train_pred_A == train["Survived"]).mean()
acc_B = (train_pred_B == train["Survived"]).mean()
print(f"Train accuracy A(gender only): {acc_A:.4f}")
print(f"Train accuracy B(gender+light rules): {acc_B:.4f}")

best = "A" if acc_A >= acc_B else "B"
print("Use baseline:", best)

# 5) test に適用して提出ファイル作成
if best == "A":
    test_pred = predict_A(test)
else:
    test_pred = predict_B(test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_pred.astype(int)
})
dst = "/kaggle/working/submission.csv"
submission.to_csv(dst, index=False)

print("作成OK?:", os.path.exists(dst), "→", dst)
print(submission.head())